In [1]:
import pandas as pd
import numpy as np
import os

from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler

from torch.utils.data import Dataset, DataLoader
from torchmetrics.classification import BinaryF1Score

from collections import Counter
import matplotlib.pyplot as plt

c:\Users\PNC\anaconda3\envs\Project_38\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
DATA_PATH = "./merged_data.csv"
LABEL_PATH = "./Label.csv"

data_df = pd.read_csv(DATA_PATH)
label_df = pd.read_csv(LABEL_PATH)

In [29]:
print(data_df.head())
print()
print()
print(label_df.head())

   frame  landmark_id         x         y         z  source_index
0      1            0  0.432988  0.559980 -0.354303             0
1      1           11  0.465422  0.526178 -0.257993             0
2      1           12  0.408942  0.558263 -0.288992             0
3      1           13  0.474997  0.550322 -0.326235             0
4      1           14  0.391634  0.598874 -0.350726             0


   source_index  label
0             0      0
1             1      0
2             2      0
3             3      0
4             4      0


In [30]:
data_df["source_index"].value_counts()


source_index
2652    5603
2655    5499
2653    5382
2651    4966
2648    4914
        ... 
1116    3900
1117    3900
1118    3900
1119    3900
3270    3900
Name: count, Length: 2901, dtype: int64

In [3]:
val = data_df["source_index"].value_counts().values
idx = data_df["source_index"].value_counts().index

for i, num in enumerate(val):
    if num > 3900:
        data_df[data_df["source_index"] == idx[i]] = data_df[data_df["source_index"] == idx[i]][:3900]

data_df["source_index"].value_counts()

source_index
0.0       3900
2175.0    3900
2177.0    3900
2178.0    3900
2179.0    3900
          ... 
1120.0    3900
1121.0    3900
1122.0    3900
1123.0    3900
3270.0    3900
Name: count, Length: 2901, dtype: int64

In [4]:
data_df.dropna(inplace = True)

In [78]:

# frame 오름차순, source_index 우선 정렬
df = data_df.sort_values(by=["source_index", "frame"])

# 피벗: (frame, source_index) 기준, landmark_id 별 x/y/z를 칼럼으로
df_pivot = df.pivot(index=["frame", "source_index"], columns="landmark_id", values=["x", "y", "z"])

# 다중 인덱스 컬럼을 단일 열로 변환: ex) ('x', 11.0) -> x_11
df_pivot.columns = [f"{coord}_{int(lid)}" for coord, lid in df_pivot.columns]

# 인덱스 복구
df_pivot.reset_index(inplace=True)

# 다시 source_index 기준으로 묶고, 그 안에서 frame 오름차순 정렬 (보장용)
df_pivot = df_pivot.sort_values(by=["source_index", "frame"]).reset_index(drop=True)

# 결과 저장
df_pivot.to_csv("10fps_reshaped_data.csv", index=False)

In [5]:
data_df = pd.read_csv("10fps_reshaped_data.csv")

In [6]:
data_df["source_index"].value_counts()

source_index
0.0       300
2175.0    300
2177.0    300
2178.0    300
2179.0    300
         ... 
1120.0    300
1121.0    300
1122.0    300
1123.0    300
3270.0    300
Name: count, Length: 2901, dtype: int64

In [52]:
data_df.head()

,frame,source_index,x_0,x_11,x_12,x_13,x_14,x_15,x_16,x_23,...,z_13,z_14,z_15,z_16,z_23,z_24,z_25,z_26,z_29,z_30
0,1.0,0.0,0.432988,0.465422,0.408942,0.474997,0.391634,0.438641,0.427603,0.450574,...,-0.326235,-0.350726,-0.429115,-0.415761,0.009984,-0.010148,0.093614,0.096314,0.285863,0.288855
1,2.0,0.0,0.432988,0.465422,0.408942,0.474997,0.391634,0.438641,0.427603,0.450574,...,-0.326235,-0.350726,-0.429115,-0.415761,0.009984,-0.010148,0.093614,0.096314,0.285863,0.288855
2,3.0,0.0,0.432988,0.465422,0.408942,0.474997,0.391634,0.438641,0.427603,0.450574,...,-0.326235,-0.350726,-0.429115,-0.415761,0.009984,-0.010148,0.093614,0.096314,0.285863,0.288855
3,4.0,0.0,0.432988,0.465422,0.408942,0.474997,0.391634,0.438641,0.427603,0.450574,...,-0.326235,-0.350726,-0.429115,-0.415761,0.009984,-0.010148,0.093614,0.096314,0.285863,0.288855
4,5.0,0.0,0.432988,0.465422,0.408942,0.474997,0.391634,0.438641,0.427603,0.450574,...,-0.326235,-0.350726,-0.429115,-0.415761,0.009984,-0.010148,0.093614,0.096314,0.285863,0.288855


In [7]:
X_list = []

for i in data_df["source_index"].unique():
    X_list.append(np.array(data_df[data_df["source_index"] == i].drop(["source_index"], axis = 1)))

X_list

[array([[1.00000000e+00, 4.32988077e-01, 4.65422332e-01, ...,
         9.63138640e-02, 2.85862595e-01, 2.88854957e-01],
        [2.00000000e+00, 4.32988077e-01, 4.65422332e-01, ...,
         9.63138640e-02, 2.85862595e-01, 2.88854957e-01],
        [3.00000000e+00, 4.32988077e-01, 4.65422332e-01, ...,
         9.63138640e-02, 2.85862595e-01, 2.88854957e-01],
        ...,
        [2.98000000e+02, 5.34453928e-01, 5.21661162e-01, ...,
         7.34806955e-02, 5.49282134e-01, 2.91739076e-01],
        [2.99000000e+02, 5.34453928e-01, 5.21661162e-01, ...,
         7.34806955e-02, 5.49282134e-01, 2.91739076e-01],
        [3.00000000e+02, 5.34453928e-01, 5.21661162e-01, ...,
         7.34806955e-02, 5.49282134e-01, 2.91739076e-01]]),
 array([[1.00000000e+00, 7.75417447e-01, 7.93765128e-01, ...,
         1.33627459e-01, 3.96434814e-02, 2.25951567e-01],
        [2.00000000e+00, 7.75417447e-01, 7.93765128e-01, ...,
         1.33627459e-01, 3.96434814e-02, 2.25951567e-01],
        [3.00000000e+00, 

In [8]:
y_list = list(label_df["label"])

y_list[-5:]

[0, 0, 0, 0, 0]

In [9]:
print(len(X_list), len(y_list))

2901 2901


In [10]:
X_train, X_test, y_train, y_test = train_test_split(X_list, y_list, 
                                                    test_size = 0.2,
                                                    random_state = 42,
                                                    stratify = y_list)

In [57]:
len(X_train)

2320

In [11]:
class DetectDataset(Dataset):
    def __init__(self, feature, target):
        self.feature = feature
        self.target = target
        self.n_rows = len(self.feature)

    def __len__(self):
        return self.n_rows
    
    def __getitem__(self, index):
        featureTS = torch.tensor(self.feature[index], dtype = torch.float32)
        targetTS = torch.tensor([self.target[index]], dtype = torch.float32)

        return featureTS, targetTS

In [12]:
BATCH_SIZE = 64

trainDS = DetectDataset(X_train, y_train)
trainDL = DataLoader(trainDS, batch_size = BATCH_SIZE, shuffle = True)

In [13]:
class FallDetectModel(nn.Module):
    def __init__(self, input_size, hidden_dim, n_layers, 
                 dropout, bidirectional):
        super().__init__()

        self.model = nn.LSTM(
            input_size = input_size,
            hidden_size = hidden_dim,
            num_layers = n_layers,
            dropout = dropout,
            bidirectional = bidirectional,
            batch_first = True
        )

        if bidirectional:
            self.output = nn.Linear(hidden_dim * 2 , 1)
        
        else:
            self.output = nn.Linear(hidden_dim, 1)
    
    def forward(self, inputs):
        output, _ = self.model(inputs)
        output = output[:, -1, :]
        result = self.output(output)
        
        return result

In [14]:
EPOCH = 1000
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
LR = 0.001

input_size = 40
hidden_dim = 64
n_layers = 2
dropout = 0
bidirectional = True

fall_detect_model = FallDetectModel(input_size = input_size, 
                                    hidden_dim = hidden_dim,
                                    n_layers = n_layers,
                                    dropout = dropout,
                                    bidirectional = bidirectional).to(DEVICE)

In [15]:
fall_detect_model

FallDetectModel(
  (model): LSTM(40, 64, num_layers=2, batch_first=True, bidirectional=True)
  (output): Linear(in_features=128, out_features=1, bias=True)
)

In [16]:
F1score = BinaryF1Score().to(DEVICE)
BCELoss = nn.BCEWithLogitsLoss()

optimizer = optim.Adam(fall_detect_model.parameters(), lr = LR)
scheduler = lr_scheduler.ReduceLROnPlateau(optimizer, mode = "max", patience = 10, verbose = True)

In [17]:
def testing(model, feature, target):
    featureTS = torch.FloatTensor(feature).to(DEVICE)
    targetTS = torch.FloatTensor(target).to(DEVICE)
    
    targetTS = targetTS.unsqueeze(1)

    model.eval()

    with torch.no_grad():
        pre_val = model(featureTS)
        loss_val = BCELoss(pre_val, targetTS)

        probs = torch.sigmoid(pre_val)
        preds = (probs > 0.5).int()

        score_val = F1score(preds, targetTS.int())
 

    return loss_val, score_val, preds

In [18]:
def training(model, trainDL, X_test, y_test):
    SAVE_PATH = './saved_models/'
    os.makedirs(SAVE_PATH, exist_ok = True)

    BREAK_CNT_SCORE = 0
    LIMIT_VALUE = 1000

    BCE_LOSS_HISTORY, SCORE_HISTORY = [[], []], [[], []]

    for epoch in range(1, EPOCH + 1):
        model.train()
        SAVE_WEIGHT = os.path.join(SAVE_PATH, f"model_weights_{epoch}.pth")

        bce_loss_total, score_total = 0, 0

        for featureTS, targetTS in trainDL:
            featureTS = featureTS.to(DEVICE)
            targetTS = targetTS.to(DEVICE)
            

            pre_val = model(featureTS)

            bce_loss = BCELoss(pre_val, targetTS)
            
            probs = torch.sigmoid(pre_val)
            preds = (probs > 0.5).int()

            score = F1score(preds, targetTS)
            
            bce_loss_total += bce_loss.item()
            score_total += score.item()

            optimizer.zero_grad()
            bce_loss.backward()
            optimizer.step()

        test_bce_loss, test_score, test_preds = testing(model, X_test, y_test)
        
        BCE_LOSS_HISTORY[0].append(bce_loss_total / len(trainDL))
        SCORE_HISTORY[0].append(score_total / len(trainDL))

        BCE_LOSS_HISTORY[1].append(test_bce_loss)
        SCORE_HISTORY[1].append(test_score)

        print(f"[{epoch} / {EPOCH}]\n - TRAIN BCE LOSS : {BCE_LOSS_HISTORY[0][-1]}")
        print(f"- TRAIN F1 SCORE : {SCORE_HISTORY[0][-1]}")

        print(f"\n - TEST BCE LOSS : {BCE_LOSS_HISTORY[1][-1]}")
        print(f"- TEST F1 SCORE : {SCORE_HISTORY[1][-1]}")

        scheduler.step(test_score)

        if len(SCORE_HISTORY[1]) >= 2:
            if SCORE_HISTORY[1][-1] <= SCORE_HISTORY[1][-2]: BREAK_CNT_SCORE += 1

        if len(SCORE_HISTORY[1]) == 1:
            torch.save(model.state_dict(), SAVE_WEIGHT)
        
        else:
            if SCORE_HISTORY[1][-1] > max(SCORE_HISTORY[1][:-1]):
                torch.save(model.state_dict(), SAVE_WEIGHT)

        if BREAK_CNT_SCORE > LIMIT_VALUE:
            print(f"성능 및 손실 개선이 없어서 {epoch} EPOCH에 학습 중단")
            break

    return BCE_LOSS_HISTORY, SCORE_HISTORY

In [66]:
bce_loss, f1_score = training(fall_detect_model, trainDL, X_test, y_test)

[1 / 1000]
 - TRAIN BCE LOSS : 0.6919840155421076
- TRAIN F1 SCORE : 0.6464447161635837

 - TEST BCE LOSS : 0.6907363533973694
- TEST F1 SCORE : 0.6929134130477905
[2 / 1000]
 - TRAIN BCE LOSS : 0.6898963564151043
- TRAIN F1 SCORE : 0.6736820237459363

 - TEST BCE LOSS : 0.6858397722244263
- TEST F1 SCORE : 0.6929134130477905
[3 / 1000]
 - TRAIN BCE LOSS : 0.6844656596312652
- TRAIN F1 SCORE : 0.5265919231885189

 - TEST BCE LOSS : 0.6808013319969177
- TEST F1 SCORE : 0.6929134130477905
[4 / 1000]
 - TRAIN BCE LOSS : 0.6557372808456421
- TRAIN F1 SCORE : 0.6412110885454191

 - TEST BCE LOSS : 0.5964450836181641
- TEST F1 SCORE : 0.7475728392601013
[5 / 1000]
 - TRAIN BCE LOSS : 0.48285359224757635
- TRAIN F1 SCORE : 0.7883038923547074

 - TEST BCE LOSS : 0.3707590401172638
- TEST F1 SCORE : 0.8839941024780273
[6 / 1000]
 - TRAIN BCE LOSS : 0.3547304351184819
- TRAIN F1 SCORE : 0.8661676790263202

 - TEST BCE LOSS : 0.32112792134284973
- TEST F1 SCORE : 0.8647686839103699
[7 / 1000]
 - 

In [111]:
for x, y in trainDL:
    print(x.shape)
    break

torch.Size([64, 300, 40])


In [20]:
video = r"./Test_Dataset/test_video2.mp4"

import cv2
import mediapipe as mp




In [21]:
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(
    static_image_mode=False,
    model_complexity=0,
    min_detection_confidence=0.7,
    min_tracking_confidence=0.7,
    smooth_landmarks=True
)

important_landmarks = [
    0, 11, 12, 13, 14, 15, 16,
    23, 24, 25, 26, 29, 30
]

test_video = pd.DataFrame(columns = ["frame", "landmark_id", "x", "y", "z"])

frame_count = 0
processed_frame = 0

cap = cv2.VideoCapture(video)
original_fps = cap.get(cv2.CAP_PROP_FPS)
skip_interval = max(1, round(original_fps / 10))

while cap.isOpened():
    ret, frame = cap.read()
    # 예: 90도 시계 방향으로 회전 (세로 영상이 눕는 경우)
    frame = cv2.rotate(frame, cv2.ROTATE_90_CLOCKWISE)

    if not ret:
        break

    if frame_count % skip_interval == 0:
        # frame = cv2.rotate(frame, cv2.ROTATE_90_CLOCKWISE)
        processed_frame += 1
        frame = cv2.resize(frame, (640, 480))
        image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = pose.process(image_rgb)

        if results.pose_landmarks:
            for idx in important_landmarks:
                landmark = results.pose_landmarks.landmark[idx]
                test_video.loc[len(test_video)] = [processed_frame, idx, landmark.x, landmark.y, landmark.z]

    frame_count += 1

cap.release()


In [22]:
test_video

,frame,landmark_id,x,y,z
0,3.0,0.0,0.495077,0.202133,-0.259547
1,3.0,11.0,0.592108,0.307687,-0.000086
2,3.0,12.0,0.399682,0.322056,-0.001780
3,3.0,13.0,0.647337,0.425211,0.014860
4,3.0,14.0,0.329309,0.407353,0.015185
...,...,...,...,...,...
1477,116.0,24.0,0.371051,0.565896,-0.008246
1478,116.0,25.0,0.573948,0.641452,-0.297894
1479,116.0,26.0,0.377047,0.724311,-0.145936
1480,116.0,29.0,0.581554,0.861283,-0.186450


In [23]:
# 전체 프레임과 랜드마크 ID 목록 생성
all_frames = np.arange(1, 301)
all_landmarks = test_video['landmark_id'].unique()

df_frames = test_video["frame"].unique()

frame_list = []
landmark_list = []

for frame in all_frames:
    if frame not in df_frames:
        for landmark in all_landmarks:
            frame_list.append(frame)
            landmark_list.append(landmark)

add_row = pd.DataFrame({"frame" : frame_list, "landmark_id" : landmark_list})

test_video = pd.concat([test_video, add_row], ignore_index = True)

test_video = test_video.groupby("landmark_id").apply(lambda x : x.sort_values(by = ["frame", "landmark_id"]).interpolate().ffill().bfill()).reset_index(drop=True)
test_video = test_video.sort_values(by = ["frame", "landmark_id"])

if len(test_video) > 3900:
    test_video = test_video[:3900]

test_video

,frame,landmark_id,x,y,z
0,1.0,0.0,0.495077,0.202133,-0.259547
300,1.0,11.0,0.592108,0.307687,-0.000086
600,1.0,12.0,0.399682,0.322056,-0.001780
900,1.0,13.0,0.647337,0.425211,0.014860
1200,1.0,14.0,0.329309,0.407353,0.015185
...,...,...,...,...,...
2699,300.0,24.0,0.371051,0.565896,-0.008246
2999,300.0,25.0,0.573948,0.641452,-0.297894
3299,300.0,26.0,0.377047,0.724311,-0.145936
3599,300.0,29.0,0.581554,0.861283,-0.186450


In [24]:
# frame 오름차순, source_index 우선 정렬
test_video = test_video.sort_values(by=["frame"])

# 피벗: (frame, source_index) 기준, landmark_id 별 x/y/z를 칼럼으로
df_pivot = test_video.pivot(index=["frame"], columns="landmark_id", values=["x", "y", "z"])

# 다중 인덱스 컬럼을 단일 열로 변환: ex) ('x', 11.0) -> x_11
df_pivot.columns = [f"{coord}_{int(lid)}" for coord, lid in df_pivot.columns]

# 인덱스 복구
df_pivot.reset_index(inplace=True)
                     
# 다시 source_index 기준으로 묶고, 그 안에서 frame 오름차순 정렬 (보장용)
df_pivot = df_pivot.sort_values(by=["frame"]).reset_index(drop=True)

df_pivot
 

,frame,x_0,x_11,x_12,x_13,x_14,x_15,x_16,x_23,x_24,...,z_13,z_14,z_15,z_16,z_23,z_24,z_25,z_26,z_29,z_30
0,1.0,0.495077,0.592108,0.399682,0.647337,0.329309,0.651901,0.310710,0.537525,0.436034,...,0.014860,0.015185,-0.122630,-0.105123,0.022949,-0.022683,-0.287023,-0.434515,-0.242776,-0.385776
1,2.0,0.495077,0.592108,0.399682,0.647337,0.329309,0.651901,0.310710,0.537525,0.436034,...,0.014860,0.015185,-0.122630,-0.105123,0.022949,-0.022683,-0.287023,-0.434515,-0.242776,-0.385776
2,3.0,0.495077,0.592108,0.399682,0.647337,0.329309,0.651901,0.310710,0.537525,0.436034,...,0.014860,0.015185,-0.122630,-0.105123,0.022949,-0.022683,-0.287023,-0.434515,-0.242776,-0.385776
3,4.0,0.493193,0.595036,0.408497,0.678957,0.333286,0.661445,0.311194,0.553189,0.450343,...,-0.037487,-0.021422,-0.183644,-0.139331,0.013685,-0.013566,-0.136108,-0.406317,0.093499,-0.230269
4,5.0,0.492759,0.595715,0.412542,0.680569,0.338718,0.664453,0.312522,0.554868,0.456708,...,-0.043242,-0.012215,-0.169574,-0.093957,0.014683,-0.014548,-0.033808,-0.260285,0.177826,-0.076468
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,296.0,0.342838,0.469944,0.267143,0.603506,0.223710,0.589624,0.249557,0.485274,0.371051,...,-0.009098,0.022492,-0.133869,-0.081422,0.007985,-0.008246,-0.297894,-0.145936,-0.186450,0.125205
296,297.0,0.342838,0.469944,0.267143,0.603506,0.223710,0.589624,0.249557,0.485274,0.371051,...,-0.009098,0.022492,-0.133869,-0.081422,0.007985,-0.008246,-0.297894,-0.145936,-0.186450,0.125205
297,298.0,0.342838,0.469944,0.267143,0.603506,0.223710,0.589624,0.249557,0.485274,0.371051,...,-0.009098,0.022492,-0.133869,-0.081422,0.007985,-0.008246,-0.297894,-0.145936,-0.186450,0.125205
298,299.0,0.342838,0.469944,0.267143,0.603506,0.223710,0.589624,0.249557,0.485274,0.371051,...,-0.009098,0.022492,-0.133869,-0.081422,0.007985,-0.008246,-0.297894,-0.145936,-0.186450,0.125205


In [25]:
model = fall_detect_model

# 2. 그런 다음 state_dict를 로드합니다
model.load_state_dict(torch.load('./Final_model/LSTM(64, 96.4, 10fps).pth'))

C:\Users\PNC\AppData\Local\Temp\ipykernel_27408\1910885452.py:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('./Final_model/LSTM(64, 96.4,

<All keys matched successfully>

In [26]:
X_tensor = torch.tensor(df_pivot.values, dtype=torch.float32).to(DEVICE)
X_tensor = X_tensor.unsqueeze(0)

In [30]:
fall_detect_model.eval()
with torch.no_grad():
    output = fall_detect_model(X_tensor)  # 출력: (batch_size, 1)
    probs = torch.sigmoid(output)         # 확률화
    predicted_classes = (probs > 0.5).int()  # 0 또는 1로 변환
print(probs)
print("예측 결과:", predicted_classes)


tensor([[0.0037]], device='cuda:0')
예측 결과: tensor([[0]], device='cuda:0', dtype=torch.int32)


In [38]:
model.eval()
model = model.to('cpu')  # 디바이스 일치

example_input = torch.randn(1, 300, 40)
traced_model = torch.jit.trace(model, example_input)
traced_model.save("./Final_model/model_script.pt")


In [40]:
print(traced_model.graph)
print(traced_model.code)

graph(%self.1 : __torch__.___torch_mangle_8.FallDetectModel,
      %inputs : Float(1, 300, 40, strides=[12000, 40, 1], requires_grad=0, device=cpu)):
  %output : __torch__.torch.nn.modules.linear.___torch_mangle_7.Linear = prim::GetAttr[name="output"](%self.1)
  %model : __torch__.torch.nn.modules.rnn.___torch_mangle_6.LSTM = prim::GetAttr[name="model"](%self.1)
  %137 : Tensor = prim::CallMethod[name="forward"](%model, %inputs)
  %99 : int = prim::Constant[value=0]() # C:\Users\PNC\AppData\Local\Temp\ipykernel_27408\485676160.py:23:0
  %100 : int = prim::Constant[value=0]() # C:\Users\PNC\AppData\Local\Temp\ipykernel_27408\485676160.py:23:0
  %101 : int = prim::Constant[value=9223372036854775807]() # C:\Users\PNC\AppData\Local\Temp\ipykernel_27408\485676160.py:23:0
  %102 : int = prim::Constant[value=1]() # C:\Users\PNC\AppData\Local\Temp\ipykernel_27408\485676160.py:23:0
  %103 : Float(1, 300, 128, strides=[128, 128, 1], requires_grad=1, device=cpu) = aten::slice(%137, %99, %100, %10

In [42]:
X_tensor = X_tensor.to('cpu')

torch.sigmoid(traced_model(X_tensor))

tensor([[0.0037]], grad_fn=<SigmoidBackward0>)